In [1]:
import os
import json

from typing import List, Dict, Any, Tuple, Optional

from formating import format_date, format_value, escape_string
from crud import insert_records, delete_records, fetch_records, get_tables

In [2]:
def convert_json_to_insert_sql(json_data: Dict[str, Any], column_mapping: List[Dict[str, str]], target_tablename:str) -> str:
    """Convert JSON data to SQL insert statement with multiple VALUES."""
    
    # Get column names
    columns = [mapping['target'] for mapping in column_mapping]
    columns_sql = ', '.join(columns)
    
    # Collect all value sets
    all_values = []
    for record in json_data:
        values = []
        for mapping in column_mapping:
            source_field = mapping['source']
            value_type = mapping['type']
            try:
                value = mapping['default']
            except KeyError:
                value = record.get(source_field)    
            formatted_value = format_value(value, value_type)
            values.append(formatted_value)
        all_values.append(f"({', '.join(str(value) for value in values)})")
    
    # Create single INSERT statement with multiple VALUES
    values_sql = ',\n'.join(all_values)
    sql_statement = f"INSERT INTO {target_tablename} ({columns_sql})\nVALUES\n{values_sql};"
    
    return sql_statement

def convert_json_to_update_sql(json_data: Dict[str, Any], column_mapping: List[Dict[str, str]], target_tablename: str, source_id: Optional[str] = None, target_id: Optional[str] = None) -> str:
    """Convert JSON data to SQL update statements."""
    
    update_statements = []
    for record in json_data:
        # Build SET clause
        set_values = []
        for mapping in column_mapping:
            source_field = mapping['source']
            target_field = mapping['target']
            value_type = mapping['type']
            
            try:
                value = mapping['default']
            except KeyError:
                value = record.get(source_field)
                
            formatted_value = format_value(value, value_type)
            set_values.append(f"{target_field} = {formatted_value}")
        
        # Create UPDATE statement for each record
        set_clause = ',\n    '.join(set_values)
        update_sql = f"""UPDATE {target_tablename}
                            SET
                                {set_clause}
                            WHERE id = {record[f'{source_id}']};"""
        update_statements.append(update_sql)
    
    # Join all UPDATE statements
    return '\n\n'.join(update_statements)

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
assets_dir = os.path.join(parent_dir, '_assets')
db_dir = os.path.join(parent_dir, 'fetusapp')
db_filename = 'data.sqlite'
data_filename = 'detoraki_fetusdata.json'
mapping_filename = 'column_mapping.json'

db_path = os.path.join(db_dir, db_filename)
json_file = os.path.join(assets_dir, data_filename)
mapping_file = os.path.join(current_dir, mapping_filename)

Load the source data

In [4]:
with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

Load the column mapping from the JSON file

In [7]:
with open(mapping_file, 'r', encoding='utf-8') as f:
        column_mapping = json.load(f)

### Setup admin in Users

The script should already be there in _assets

In [13]:
target_tablename = 'users'
insert_records(target_tablename, db_path)


In [ ]:
# fetch_records(target_tablename, db_path)

In [11]:
# delete_records(target_tablename, db_path)

### Admin user created
--------------------------
### Migrate Patients

Generate the sql output and save it

In [ ]:
source_tablename = 's_org_ext'
target_tablename = 'patients'
data_index = 3

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
patient_data = json_data[data_index]['data']
null_lastname_ids = [item['s_org_extID'] for item in patient_data if item['lastName'] == ""]
print(null_lastname_ids)

In [30]:
delete_records(target_tablename, db_path)

In [32]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

---------------------

### Migrate MedicalHistory

In [21]:
source_tablename = 'tblmedhistory'
target_tablename = 'history_medical'
data_index = 10

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [19]:
insert_records(target_tablename, db_path)

In [18]:
# delete_records(target_tablename, db_path)

In [20]:
fetch_records(target_tablename, db_path)

(2, 3, 1, 1, '2013-08-26 00:00:00', '2022-12-19 21:51:42', 'Έχει κάνει για τα κρεατάκια', 'Ναι', 'Ναι', 'Καταναλώνει ένα ποτηράκι κρασί την ημέρα', 0, 'Δεν έχει παρατηρηθεί καμία αντίδραση', 1, 'Στην ασπιρίνη', 'Στην στριχνίνη', 'Υπάρχει κάτι αλλά δεν θέλει να πει τι.', 53, 1.63, 19.948, 'Hi Sveta,\n\nWorking-wise everything is great. I still enjoy working for Close, both in terms of the projects I am involved with as well as the role. Unfortunately I have not registered for the CFA exam in July (may try next December instead). I am splitting my time between London and Athens (currently going back every second weekend) cause my mum’s condition has deteriorated rapidly so I can’t commit to studying over the weekends as much as I would like.\n\nBut Carl has sent out an invite and has started a working group for BAs and Big Data so I can at least hit one of my objectives for this semester. Wasn’t able to join his first meeting on Monday (09/11) because I was still in Athens but I will do 

------------------

### Migrate Pathology data

In [ ]:
fetch_records(target_tablename, db_path)

In [10]:
get_tables(db_path)

('alembic_version',)
('users',)
('patients',)
('history_medical',)
